# 리뷰완료

이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 코사인 유사도 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/24603

# 1. 코사인 유사도(Cosine Similarity)

In [1]:
from numpy import dot # 내적 (Dot product, Inner product)
from numpy.linalg import norm # 선형대수 함수(Linear Algebra)
import numpy as np

In [2]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [3]:
# Norm
# Norm 은 벡터의 크기(혹은 길이)를 측정하는 방법(혹은 함수)이다.
# 또는 두 벡터 사이의 거리를 측정하는 방법이기도 하다.

In [4]:
A = np.array([0,1,1,1])
norm(A) # 루트 (0의 제곱 + 1의 제곱 + 1의 제곱 + 1의 제곱) = √3 = 1.732

1.7320508075688772

In [5]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

In [6]:
print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


# 2. 유사도를 이용한 추천 시스템 구현하기

In [7]:
!pip install --user numpy==1.23.0

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
data = pd.read_csv('./dataset/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [12]:
data.shape

(45466, 24)

In [ ]:
data = data.head(20000)

In [13]:
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 954


In [14]:
data['overview'] = data['overview'].fillna('')

In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (45466, 75827)


In [26]:
# tfidf 
# TfidfVectorizer 로 tfidf 계산 결과 테이블을 만들어준다.
# tfidf = Tf와 idf 의 곱이다.

# 1. Tf(Term Frequency) : 하나의 문서(문장)에서 특정 단어가 등장하는 횟수
# 2. Idf(Inverse Document Frequency) : Df(Document Frequency)는 문서 빈도. 특정 단어가 몇 개의 문서(문장)에서 등장하는지를 수치화 한 것.
# 그것의 역수가 idf다. 보통 그냥 역수를 취하기 보다는 아래처럼 수식화한다. 역수 개념을 사용하는 이유는, 적은 문서(문장)에 등장할수록 큰 숫자가 되게하고 반대로 많은 문서(문장)에 등장할수록 
# 숫자를 작아지게 함으로써 여러 문서(문장)에 의미 없이 사용되는 단어의 가중치를 줄이기 위해서다.

# Tf-idf 수치는 Tf 값과 Idf 값을 곱하여 구한다. 해당 연산을 거친 최종 Tf-idf 값은 0과 1사이로 만들어진다.

In [27]:
tfidf

TfidfVectorizer(stop_words='english')

In [25]:
tfidf_matrix

<45466x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [16]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [17]:
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (45466, 45466)


In [18]:
title_to_index = dict(zip(data['title'], data.index))

In [23]:
title_to_index

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 29042,
 'Sabrina': 888,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'The American President': 10,
 'Dracula: Dead and Loving It': 11,
 'Balto': 12,
 'Nixon': 13,
 'Cutthroat Island': 14,
 'Casino': 15,
 'Sense and Sensibility': 41042,
 'Four Rooms': 17,
 'Ace Ventura: When Nature Calls': 18,
 'Money Train': 19,
 'Get Shorty': 20,
 'Copycat': 21,
 'Assassins': 22,
 'Powder': 23,
 'Leaving Las Vegas': 24,
 'Othello': 21274,
 'Now and Then': 26,
 'Persuasion': 40837,
 'The City of Lost Children': 28,
 'Shanghai Triad': 29,
 'Dangerous Minds': 30,
 'Twelve Monkeys': 31,
 'Wings of Courage': 32,
 'Babe': 33,
 'Carrington': 34,
 'Dead Man Walking': 35,
 'Across the Sea of Time': 36,
 'It Takes Two': 29129,
 'Clueless': 38,
 'Cry, the Beloved Country': 26667,
 'Richard III': 17719,
 'Dead Presidents': 41,
 'Restoration': 38571,
 'Mortal Kombat': 43,
 'To D

In [19]:
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [20]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [idx[0] for idx in sim_scores]

    return data['title'].iloc[movie_indices]

In [21]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object